In [7]:
from pathlib import Path
import html
import mimetypes
import argparse

IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".gif", ".bmp", ".svg"}

HTML_TEMPLATE = """<!doctype html>
<html lang="zh-CN">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>{title}</title>
  <style>
    :root {{
      --gap: 12px;
      --thumb-h: 180px;
      --bg: #0b0c10;
      --card: #111218;
      --text: #e8e8ea;
      --muted: #a9abb3;
      --border: rgba(255,255,255,0.10);
    }}
    body {{
      margin: 0;
      font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, "Helvetica Neue", Arial;
      background: var(--bg);
      color: var(--text);
    }}
    header {{
      position: sticky;
      top: 0;
      backdrop-filter: blur(8px);
      background: rgba(11,12,16,0.75);
      border-bottom: 1px solid var(--border);
      padding: 12px 16px;
      z-index: 10;
    }}
    .title {{
      font-size: 16px;
      font-weight: 600;
      margin: 0 0 6px 0;
    }}
    .meta {{
      font-size: 12px;
      color: var(--muted);
      display: flex;
      gap: 14px;
      flex-wrap: wrap;
    }}
    .wrap {{
      padding: 16px;
      max-width: 1400px;
      margin: 0 auto;
    }}
    .grid {{
      display: grid;
      grid-template-columns: repeat(auto-fill, minmax(220px, 1fr));
      gap: var(--gap);
    }}

    /* Card as button */
    .cardbtn {{
      appearance: none;
      border: none;
      background: transparent;
      padding: 0;
      text-align: left;
      cursor: zoom-in;
      color: inherit;
    }}

    .card {{
      background: var(--card);
      border: 1px solid var(--border);
      border-radius: 14px;
      overflow: hidden;
      box-shadow: 0 6px 20px rgba(0,0,0,0.25);
      transition: transform .12s ease;
    }}
    .cardbtn:hover .card {{ transform: translateY(-2px); }}

    .thumb {{
      height: var(--thumb-h);
      display: flex;
      align-items: center;
      justify-content: center;
      background: #0f1016;
    }}
    .thumb img {{
      max-width: 100%;
      max-height: 100%;
      object-fit: contain;
      display: block;
      user-select: none;
      -webkit-user-drag: none;
    }}
    .info {{
      padding: 10px 12px 12px 12px;
      border-top: 1px solid var(--border);
    }}
    .name {{
      font-size: 12px;
      line-height: 1.35;
      word-break: break-all;
    }}
    .tag {{
      margin-top: 6px;
      font-size: 11px;
      color: var(--muted);
    }}

    .footer {{
      padding: 18px 0 10px 0;
      text-align: center;
      color: var(--muted);
      font-size: 12px;
    }}

    /* Lightbox */
    .lightbox {{
      position: fixed;
      inset: 0;
      display: none;
      z-index: 9999;
    }}
    .lightbox.open {{ display: block; }}
    .lb-backdrop {{
      position: absolute;
      inset: 0;
      background: rgba(0,0,0,0.75);
    }}
    .lb-panel {{
      position: absolute;
      inset: 24px;
      display: grid;
      grid-template-rows: auto 1fr auto;
      gap: 10px;
    }}
    .lb-top {{
      display: flex;
      justify-content: space-between;
      align-items: center;
      gap: 12px;
      color: var(--text);
      font-size: 13px;
    }}
    .lb-title {{
      overflow: hidden;
      text-overflow: ellipsis;
      white-space: nowrap;
      color: var(--muted);
    }}
    .lb-actions {{
      display: flex;
      gap: 8px;
      flex-shrink: 0;
    }}
    .lb-btn {{
      border: 1px solid var(--border);
      background: rgba(17,18,24,0.8);
      color: var(--text);
      border-radius: 10px;
      padding: 8px 10px;
      cursor: pointer;
      font-size: 12px;
    }}
    .lb-btn:hover {{ filter: brightness(1.1); }}

    .lb-stage {{
      position: relative;
      border: 1px solid var(--border);
      background: rgba(17,18,24,0.5);
      border-radius: 16px;
      overflow: hidden;
      display: flex;
      align-items: center;
      justify-content: center;
    }}
    .lb-img {{
      max-width: 100%;
      max-height: 100%;
      object-fit: contain;
      cursor: zoom-out;
      user-select: none;
      -webkit-user-drag: none;
    }}
    .lb-bottom {{
      display: flex;
      justify-content: center;
      gap: 10px;
      color: var(--muted);
      font-size: 12px;
    }}
    .lb-hint {{
      opacity: 0.9;
    }}
    body.lb-lock {{
      overflow: hidden;
    }}
  </style>
</head>
<body>
<header>
  <div class="title">{title}</div>
  <div class="meta">
    <div>目录：{dir_name}</div>
    <div>图片数：{count}</div>
  </div>
</header>

<div class="wrap">
  <div class="grid">
    {cards}
  </div>
  <div class="footer">Generated by Python</div>
</div>

<!-- Lightbox -->
<div id="lightbox" class="lightbox" aria-hidden="true">
  <div class="lb-backdrop" id="lb-backdrop"></div>
  <div class="lb-panel" role="dialog" aria-modal="true" aria-label="Image viewer">
    <div class="lb-top">
      <div class="lb-title" id="lb-title"></div>
      <div class="lb-actions">
        <button class="lb-btn" id="lb-prev" type="button">← 上一张</button>
        <button class="lb-btn" id="lb-next" type="button">下一张 →</button>
        <button class="lb-btn" id="lb-close" type="button">关闭 (Esc)</button>
      </div>
    </div>

    <div class="lb-stage">
      <img id="lb-img" class="lb-img" alt="">
    </div>

    <div class="lb-bottom">
      <div class="lb-hint">提示：左右方向键切换，Esc 关闭，点击图片/黑色背景关闭</div>
    </div>
  </div>
</div>

<script>
(function() {{
  const items = Array.from(document.querySelectorAll('[data-lb="item"]'));
  const lightbox = document.getElementById('lightbox');
  const imgEl = document.getElementById('lb-img');
  const titleEl = document.getElementById('lb-title');
  const btnClose = document.getElementById('lb-close');
  const btnPrev = document.getElementById('lb-prev');
  const btnNext = document.getElementById('lb-next');
  const backdrop = document.getElementById('lb-backdrop');

  let idx = -1;

  function setIndex(newIdx) {{
    if (!items.length) return;
    idx = (newIdx + items.length) % items.length;
    const el = items[idx];
    const src = el.getAttribute('data-src');
    const name = el.getAttribute('data-name') || '';
    imgEl.src = src;
    imgEl.alt = name;
    titleEl.textContent = `${{idx + 1}} / ${{items.length}}  ·  ${{name}}`;
  }}

  function openAt(i) {{
    setIndex(i);
    lightbox.classList.add('open');
    lightbox.setAttribute('aria-hidden', 'false');
    document.body.classList.add('lb-lock');
  }}

  function close() {{
    lightbox.classList.remove('open');
    lightbox.setAttribute('aria-hidden', 'true');
    document.body.classList.remove('lb-lock');
    // 避免大图占用太久内存
    imgEl.removeAttribute('src');
  }}

  function prev() {{ setIndex(idx - 1); }}
  function next() {{ setIndex(idx + 1); }}

  items.forEach((el, i) => {{
    el.addEventListener('click', () => openAt(i));
  }});

  btnClose.addEventListener('click', close);
  backdrop.addEventListener('click', close);
  imgEl.addEventListener('click', close);
  btnPrev.addEventListener('click', prev);
  btnNext.addEventListener('click', next);

  window.addEventListener('keydown', (e) => {{
    if (!lightbox.classList.contains('open')) return;
    if (e.key === 'Escape') close();
    else if (e.key === 'ArrowLeft') prev();
    else if (e.key === 'ArrowRight') next();
  }});
}})();
</script>

</body>
</html>
"""

CARD_TEMPLATE = """
<button type="button" class="cardbtn" data-lb="item" data-src="{src}" data-name="{name}">
  <div class="card">
    <div class="thumb">
      <img src="{src}" loading="lazy" alt="{alt}">
    </div>
    <div class="info">
      <div class="name">{name}</div>
      <div class="tag">{size}</div>
    </div>
  </div>
</button>
"""

def human_size(num_bytes: int) -> str:
    units = ["B", "KB", "MB", "GB", "TB"]
    size = float(num_bytes)
    i = 0
    while size >= 1024 and i < len(units) - 1:
        size /= 1024.0
        i += 1
    if i == 0:
        return f"{int(size)} {units[i]}"
    return f"{size:.2f} {units[i]}"

def is_image(p: Path) -> bool:
    if p.suffix.lower() in IMG_EXTS:
        return True
    mt, _ = mimetypes.guess_type(str(p))
    return (mt or "").startswith("image/")

def generate_gallery_html(img_dir: Path, out_html: Path, title: str | None = None) -> None:
    img_dir = img_dir.resolve()
    out_html = out_html.resolve()
    title = title or f"Image Gallery - {img_dir.name}"

    images = [p for p in img_dir.iterdir() if p.is_file() and is_image(p)]
    images.sort(key=lambda p: p.name.lower())

    cards = []
    for p in images:
        rel = p.relative_to(out_html.parent)
        rel_str = str(rel).replace("\\", "/")
        stat = p.stat()
        cards.append(CARD_TEMPLATE.format(
            src=html.escape(rel_str),
            alt=html.escape(p.name),
            name=html.escape(p.name),
            size=human_size(stat.st_size),
        ))

    out_html.parent.mkdir(parents=True, exist_ok=True)
    page = HTML_TEMPLATE.format(
        title=html.escape(title),
        dir_name=html.escape(str(img_dir)),
        count=len(images),
        cards="\n".join(cards),
    )
    out_html.write_text(page, encoding="utf-8")

In [8]:
img_dir = Path("E:\\project\\cloud_seg_compare_page\\20260126")
out_html = Path("20260126.html")
title = '20260126 Cloud Segmentation Results'

if not img_dir.exists() or not img_dir.is_dir():
    raise SystemExit(f"Not a directory: {img_dir}")

generate_gallery_html(img_dir, out_html, title=title)
print(f"✅ Generated: {out_html.resolve()}")


✅ Generated: E:\project\cloud_seg_compare_page\20260126.html
